Данная работа является продолжением серии работ с общим названием ERNIE (см. [github-репозиторий](https://github.com/PaddlePaddle/ERNIE/blob/develop/README.en.md)):
- [Sun et al., 2019]($ERNIE: Enhanced Representation through Knowledge Integration$). *ERNIE: Enhanced Representation through Knowledge Integration.*
- [Sun et al., 2019]($ERNIE 2.0: A Continual Pre-training Framework for Language Understanding$). *ERNIE 2.0: A Continual Pre-training Framework for Language Understanding.*
- [Xiao et al., 2020]($ERNIE-GEN: An Enhanced Multi-Flow Pre-training and Fine-tuning Framework for Natural Language Generation$). *ERNIE-GEN: An Enhanced Multi-Flow Pre-training and Fine-tuning Framework for Natural Language Generation.*
- [Yu et al., 2020]($ERNIE-ViL: Knowledge Enhanced Vision-Language Representations Through Scene Graph$). *ERNIE-ViL: Knowledge Enhanced Vision-Language Representations Through Scene Graph.*
- [Li et al., 2020]($UNIMO: Towards Unified-Modal Understanding and Generation via Cross-Modal Contrastive Learning$). *UNIMO: Towards Unified-Modal Understanding and Generation via Cross-Modal Contrastive Learning.*
- [Ouyang et al., 2020]($ERNIE-M: Enhanced Multilingual Representation by Aligning Cross-lingual Semantics with Monolingual Corpora$). *ERNIE-M: Enhanced Multilingual Representation by Aligning Cross-lingual Semantics with Monolingual Corpora.*
- [Xiao et al., 2020]($ERNIE-Gram: Pre-Training with Explicitly N-Gram Masked Language Modeling for Natural Language Understanding$). *ERNIE-Gram: Pre-Training with Explicitly N-Gram Masked Language Modeling for Natural Language Understanding.*
- [Ding et al., 2020]($ERNIE-Doc: A Retrospective Long-Document Modeling Transformer$). *ERNIE-Doc: A Retrospective Long-Document Modeling Transformer.*
- [Sun et al., 2021](https://arxiv.org/abs/2107.02137). *ERNIE 3.0: Large-scale Knowledge Enhanced Pre-training for Language Understanding and Generation.*

<img src="assets/ERNIE_milestone_20210519_en.png" width="800" align="center">

### GPT и BERT: сравнение и объединение

В модели ERNIE 3.0 авторы объединяют два подхода к построению и обучению языковых моделей:

1. Auto-regressive, или left-to-right language model (GPT, [Radford et al., 2018](https://s3-us-west-2.amazonaws.com/openai-assets/research-covers/language-unsupervised/language_understanding_paper.pdf))
2. Auto-encoding, или bidirectional language model (BERT, [Devlin et al., 2018](https://arxiv.org/abs/1810.04805))

Авторы сравнивают плюсы и минусы GPT и BERT. По своей архитектуре BERT плохо подходит для генерации текстов (*natural language generation*): процесс генерации сложнее, а качество генерации обычно получается хуже, чем у GPT ([Wang and Cho, 2019]($BERT has a Mouth, and It Must Speak: BERT as a Markov Random Field Language Model$)). Однако считается, что GPT хуже подходит для решения задач, связанных не пониманием текста (*natural language understanding*).

> ...most large-scale models are trained in an auto-regressive way, but [Devlin et al., 2018](https://arxiv.org/abs/1810.04805) shows that such models demonstrate poorer performance with traditional fine-tuning when adapting to downstream language understanding tasks.

Для того, чтобы модель ERNIE 3.0 могла качественно решать задачи как понимания, так и генерации текстов, авторы решили объединить оба подхода.

> ERNIE 3.0 ... combines auto-regressive network and auto-encoding network so that the trained model can handle both natural language understanding and generation tasks through zero-shot learning, few-shot learning or fine-tuning.

Гибридные языковые модели предлагались и ранее. Например, в XLNet ([Yang et al., 2019]($XLNet: Generalized Autoregressive Pretraining for Language Understanding$)) предлагается метод permutation language modeling, который реализуется с помощью модификации масок внимания; в UniLM ([Dong et al., 2019]($Unified Language Model Pre-training for Natural Language Understanding and Generation$)) предлагается обучать сеть параллельно в трех разных режимах: left-to-right, bidirectional и seq-to-seq, задавая соответствующие маски внимания.

Архитектура ERNIE 3.0 состоит из трех блоков:
- *Universal representation module* ($U$)
- *Language understanding network* ($LU$)
- *Language generation network* ($LG$)

<img src="assets/ernie3.jpg" width="700" align="center">

Каждый блок основан на [Transformer-XL]($Transformer-XL: Attentive Language Models Beyond a Fixed-Length Context$) ([Dai et al., 2019](https://arxiv.org/abs/1901.02860)), обладающим механизмом памяти и способном учитывать большую длину контекста. Механизм рекурретности при этом модифицируется так, как предложено в ERNIE-Doc ([Ding et al., 2020]($ERNIE-Doc: A Retrospective Long-Document Modeling Transformer$)). Пусть некий текст разделен на несколько фрагментов. Отличие ERNIE-Doc от Transformer-XL состоит в следующем. На $n$-м слое $t$-го фрагмента текста в качестве keys и values используются не только векторы $(n-1)$-го слое $t$-го фрагмента, но и:

- В Transformer-XL: векторы $(n-1)$-го слоя $(t-1)$-го фрагмента
- В ERNIE-Doc: векторы $n$-го слоя $(t-1)$-го фрагмента

При этом в ERNIE-Doc (и по-видимому в ERNIE 3.0) по всем фрагментам текста делается два прохода, первая фаза называется skimming, вторая - retrospective. Таким образом обеспечивается, что в фазе retrospective каждый фрагмент может использовать информацию обо всем тексте в целом (по крайней мере теоретически; на практике, как мне кажется, информация может экспоненциально забываться, особенно учитывая, что градиент не распространяется между фрагментами).

<img src="assets/erniedoc.jpg" width="700" align="center">

Если решается задача генерации текста, то модель ERNIE 3.0 работает как $LG(U(input))$, и используется треугольная маска внимания. Если же решается какая-либо другая задача, то модель работает как $LU(U(input))$, и маска внимания не используется.

Авторы отмечают, что память в Transformer-XL может использоваться только при решении задач генерации текста (что довольно странно, поскольку технически ее можно использовать и в задаче masked language model).

> And what needs special attention is that the memory module is only valid for natural language generation tasks while controlling the attention mask matrices.

Обоснование выбора именно такой архитектуры для ERNIE 3.0, хоть и присутствует, но кажется недостаточным по двум причинам:

1. Хотя GPT-подобные модели показывают меньшую точность при файн-тюнинге на задачах NLU, чем BERT-подобные модели но для них был предложен метод [P-tuning]($GPT Understands, Too$) ([Liu et al., 2021](https://arxiv.org/abs/2103.10385)), при использовании которого они, наоборот, опережают BERT-подобные модели.

2. Авторы отказываются от seq-to-seq предобучения как от излишне сложного, хотя на моделях T5 ([Raffel et al., 2019]($Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer$)), T0 ([Sanh et al., 2021]($Multitask Prompted Training Enables Zero-Shot Task Generalization$)), OFA ([Wang et al., 2022]($Unifying Architectures, Tasks, and Modalities Through a Simple Sequence-to-Sequence Learning Framework$)) и других было показано, что именно такой метод предобучения эффективен и более универсален.

> Generative pre-training models usually utilize traditional language model (such as GPT, GPT-2) or sequence-to-sequence language model (such as BART, T5, ERNIE-GEN) as the pre-training task, the latter trains on the network with an auxiliary decoder structure. *ERNIE 3.0 opt for traditional language model as the pre-training task to abate the network complexity and heighten the effectiveness of unified pre-training.*

### Использование графов знаний

Языковые модели иногда обучают для того, чтобы работать с ними как с базами знаний ([Petroni et al., 2019]($Language Models as Knowledge Bases?$)). В ходе предобучения модель накапливает различные знания о мире, и иногда исследователи хотят повысить ее способность накапливать и использовать знания с помощью дополнительных методов:

- [Sun et al., 2019]($ERNIE: Enhanced Representation through Knowledge Integration$) (ERNIE 1.0) модифицируют задачу masked language modeling, маскируя сущности в тексте целиком, вместо маскирования их случайных частей.
- [Xiong et al., 2019]($Pretrained Encyclopedia: Weakly Supervised Knowledge-Pretrained Language Model$) (WKLM) выполняют случайную замену сущностей в тексте и обучают модели распознавать такие замены.
- [Wang et al., 2019]($KEPLER: A Unified Model for Knowledge Embedding and Pre-trained Language Representation$) (KEPLER) параллельно с masked language modeling оптимизируют еще одну функцию потерь, для чего берутся две случайные сущности $A$, $B$ и связь $r$ между ними из графа знаний (например, $A$="Johannes Kepler", $r$="Occupation", $B$="astronomer"). Обозначив за $e$ энкодер трансформера, за $d(A)$, $d(B)$ определение понятий $A$ и $B$ из энциклопедии, за $\textit{emb}$ эмбеддинг, соответствующий типу связи $r$ в графе знаний и за $\langle s \rangle$ дополнительный токен, функция потерь специального вида рассчитывается на следующей тройке: $e(\langle s \rangle, d(A))$, $\textit{emb}(r)$, $e(\langle s \rangle, d(B))$.
- [Sun et al., 2020]($CoLAKE: Contextualized Language and Knowledge Embedding$) (CoLAKE) извлекают из текстов именованные сущности и ищут информацию о них в графе знаний, после чего составляют *word-knowledge graph*, подграфом которого является полносвязный граф слов в тексте. Далее энкодер трансформера работает с этим графом, при этом связность вершин в графе выражается в виде маски self-attention.
- [Zhou et al., 2020]($Pre-training Text-to-Text Transformers for Concept-centric Common Sense$) (CALM) обучают seq-to-seq трансформер восстанавливать правильный порядок слов в предложении и отличать правильные предложения от предложений с замененными словами. Этот же трансформер генерирует для себя обучающие примеры, то есть работает как генератор и как дискриминатор.
- [Zhang et al., 2022]($GreaseLM: Graph REASoning Enhanced Language Models for Question Answering$) (GreaseLM) обучают модель, которая принимает в качестве входных данных вопрос и участок графа знаний, содержащий те понятия, которые упоминаются в вопросе. Такой принцип работы похож на CoLAKE, но архитектура модели здесь сложнее: токены текста и элементы графа знаний обрабатываются по-разному.

В обучении ERNIE 3.0 также используются графы знаний. Описанные выше модели имеют средний размер (12-24 слоев трансформера), тогда как авторы ERNIE 3.0 хотят протестировать использование графов знаний при обучении более крупной модели.

ERNIE 3.0 обучается на нескольких задачах:

**1. Knowledge masked language modeling.** Аналогично ERNIE 1.0, маскируются целиком отдельные сущности (Entity-level Masking) или фразы (Phrase-level Masking). В ERNIE 1.0 на первой стадии обучения использовалось обычное маскирование случайных токенов, на второй стадии - маскирование фраз, на третьей стадии - маскирование сущностей. Для поиска сущностей и фраз используются различные средства, специфичные для языка.

> Phrase is a small group of words or characters together acting as a conceptual unit. For English, we use lexical analysis and chunking tools to get the boundary of phrases in the sentences ([Sun et al., 2019]($ERNIE: Enhanced Representation through Knowledge Integration$))

<img src="assets/knowledgemasking.jpg" width="700" align="center">

**2. Document language modeling.** Задача предсказания следующего слова в тексте (generative pre-training), при которой по очереди обрабатываются все фрагменты документа с использованием рекуррентности по аналогии с ERNIE-Doc.

**3. Sentence reordering.** Входными данными являются $k$ сегментов текста, перемешанных случайным образом, и задачей модели является определение правильного порядка. Рассматривается как задача классификации на $k!$ классов, где каждый класс соответствует определенному порядку сегментов.

**4. Sentence distance.** Входными данными являются 2 сегмента текста, и задачей модели является определить, следуют ли сегменты друг за другом в исходном тексте, или если нет, то взяты ли они из одного и того же текста. Рассматривается как задача классификации на 3 класса.

**5. Universal knowledge-text prediction.** Входными данными является связь из графа знаний, связывающая два понятия, и предложение из энциклопедии, содержащее эти же два понятия (как показано на изображении). Маскируется либо связь между понятиями, либо случайное слово из предложения. Задачей модели является определить слова, закрытые маской.

<img src="assets/knowledgemasking2.jpg" width="700" align="center">

> To incorporate knowledge into one pre-trained language model, we introduce universal knowledge-text prediction task ... To predict the relation in the triple, the model needs to detect mentions of head entity and tail entity and determine semantic relationship that holds between them in the corresponding sentence.

Из перечисленных задач только задача Document language modeling выполняется авторегрессивно.

> ERNIE 3.0 trains the **NLU network** through **knowledge masked language modeling** to improve the capacity of capturing the lexical information, trains the **sentence reordering task** and the **sentence distance discerning** task to strengthen the ability of capturing the syntactic information, and finally optimizes the model with the **universal knowledge-text prediction task** to improve knowledge memorization and reasoning. Meanwhile, ERNIE 3.0 trains the **NLG network** with the **document language modeling task** to enable various generation styles.

...TODO